In [2]:
import pandas as pd
import numpy as np
from models.MKNN import ModifiedKNN
import neattext.functions as nfx
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.metrics import f1_score, recall_score
from heapq import nsmallest as nMin
from tqdm import tqdm
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
df = pd.read_csv('Twitter_fresh/twitter_crawling.csv',encoding='latin1', usecols=['date','text'])
df.head()

# Preprocessing

In [ ]:
def casefolding(Text):
    Text = Text.lower()
    return Text

In [ ]:
df['text'] = df['text'].apply(casefolding)
df.head()

In [ ]:
def punc_clean(Text):
        Text = nfx.remove_urls(Text)
        Text = nfx.remove_punctuations(Text)
        Text = nfx.remove_emojis(Text)
        Text = nfx.remove_special_characters(Text)
        Text = nfx.remove_numbers(Text)
        return Text

In [ ]:
df['text'] = df['text'].apply(punc_clean)
df.head()

In [ ]:
def word_tokenize_wrapper(Text):
        return word_tokenize(Text)

In [ ]:
df['text'] = df['text'].apply(word_tokenize_wrapper)
df.head()

In [ ]:
def word_norm(tweets):
    word_dict = pd.read_csv('data/indonesia_slangWords.csv')
    norm_word_dict = {}
    for index, row in word_dict.iterrows():
        if row[0] not in norm_word_dict:
            norm_word_dict[row[0]] = row[1]
    return [norm_word_dict[term] if term in norm_word_dict else term for term in tweets]

In [ ]:
df['text'] = df['text'].apply(word_norm)
df.head()

In [ ]:
def remove_stopword(Text):
    stopW = stopwords.words('indonesian', 'english')
    sw = pd.read_csv('data/stopwordbahasa.csv')
    stopW.extend(sw)
    remove_sw = ' '.join(Text)
    clean_sw = [word for word in remove_sw.split() if word.lower() not in stopW]
    return clean_sw

In [ ]:
df['text'] = df['text'].apply(remove_stopword)
df.head()

In [ ]:
def indo_stem(Text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    result = []
    for w in Text:
        result.append(stemmer.stem(w))
        result.append(" ")
    return " ".join(result)

In [ ]:
df['text'] = df['text'].apply(indo_stem)
df.head()

# Sentiment

In [ ]:
def Sentiment_analysis():
    df = pd.read_csv('Twitter_fresh/Clean Data/Twitter_clean_2000.csv', encoding='utf-8')

    # LexiconVader dengan custom Lexicon(bahasa indonesia)
    sia1A, sia1B = SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer()
    senti = SentimentIntensityAnalyzer()
    # Hapus Default lexicon VADER
    sia1A.lexicon.clear()
    sia1B.lexicon.clear()
    senti.lexicon.clear()

    # Read custom Lexicon Bahasa Indonesia
    data1A = open('data/lexicon_sentimen_negatif.txt', 'r').read()
    data1B = open('data/lexicon_sentimen_positif.txt', 'r').read()
    data_senti = open('data/sentiwords_id.txt', 'r').read()
    
    # convert lexicon to dictonary
    insetNeg = json.loads(data1A)
    insetPos = json.loads(data1B)
    sa = json.loads(data_senti)

    # update lexicon vader with custom lexicon (b.indo)
    sia1A.lexicon.update(insetNeg)
    sia1B.lexicon.update(insetPos)
    senti.lexicon.update(sa)

    # method untuk cek apa sentimen pos,neg,neu
    def is_positive_inset(Text: str) -> bool:
        return sia1A.polarity_scores(Text)["compound"] + sia1B.polarity_scores(Text)["compound"] >= 0.05
    
    tweets = df['text'].to_list()

    with open('output/Sentiment-result.txt', 'w+') as f:
        for tweet in tweets:
            label = "Positive" if is_positive_inset(tweet) else "Negative"
            f.write(str(label + "\n"))

    sen = pd.read_csv('output/Sentiment-result.txt', names=['Sentiment'])
    df = df.join(sen)

    ## Save clean Dataset
    #df.to_csv('CleanText_Sentiment.csv', index=False)
    return df

In [ ]:
sentiment_result = Sentiment_analysis()

In [ ]:
sentiment_result['Sentiment'].value_counts()

In [ ]:
sentiment_result.to_csv('output/Sentiment_result.csv', index=False)

# Balance Data

In [ ]:
data = pd.read_csv('output/sentiment_result_2000.csv', encoding='utf-8')
data.head()

In [ ]:
Xfeauter = data['text']
ylabel = data['Sentiment']
print(Xfeauter.shape, ylabel.shape)

In [ ]:
tf = TfidfVectorizer(decode_error='replace')
text_tf = tf.fit_transform(Xfeauter)

In [ ]:
plt.pie(ylabel.value_counts(),labels=['Label 0 (Negative Tweets)', 'Label 1 (Positive Tweets)'],autopct='%0.1f%%')
plt.axis('equal')
plt.show()

In [ ]:
lbl = LabelEncoder()
label_cv = lbl.fit_transform(ylabel)
print(label_cv)

In [ ]:
smt = SMOTE()
text_smote, label_smote = smt.fit_resample(text_tf, label_cv)
print(text_smote.shape, label_smote.shape)

In [ ]:
plt.pie(pd.value_counts(label_smote),labels=['Label 0 (Negative Tweets)', 'Label 1 (Positive Tweets)'],autopct='%0.1f%%')
plt.axis('equal')
plt.show()

# Models Algorithm M KNN

In [ ]:
def TFIDF_word_weight(vect, word_weight):
    feature_name = np.array(vect.get_feature_names_out())
    data = word_weight.data
    indptr = word_weight.indptr
    indices = word_weight.indices
    n_docs = word_weight.shape[0]

    word_weght_list = []
    for i in range(n_docs):
        doc = slice(indptr[i], indptr[i + 1])
        count, idx = data[doc], indices[doc]
        feature = feature_name[idx]
        word_weght_dict = dict(dict(zip(feature, count)))
        word_weght_list.append(word_weght_dict)
    word_weght_df = pd.DataFrame(word_weght_list)
    word_weght_df = word_weght_df.fillna(0)
    return word_weght_df

In [ ]:
tf = TfidfVectorizer(decode_error="replace")

In [ ]:
Xfeatures = sentiment_result.text
Xfeatures = tf.fit_transform(Xfeatures)
df_tfidf = TFIDF_word_weight(tf, Xfeatures)

In [ ]:
df_tfidf.to_csv('output/tfidf_res.csv', index=False)

In [ ]:
def jarak_euc(x,y):
    return euclidean_distances(x,y)

In [ ]:
X_train = jarak_euc(Xfeatures,Xfeatures)
print(X_train)

In [ ]:
dd = pd.DataFrame(X_train)
dd.to_csv('euc.csv', index=False)

In [3]:
sentiment_result = pd.read_csv('Twitter_fresh/Balance/1000_DATA.csv', encoding='utf-8')

In [4]:
sentiment_result.shape

(1000, 5)

In [6]:
print(sentiment_result['Sentiment'].value_counts())

Negative    560
Positive    440
Name: Sentiment, dtype: int64


In [127]:
k_value = 9 # 1 - 25
X = sentiment_result['text'].values
y = sentiment_result['Sentiment'].values
fold_i = 1
fold_n = 10 # 3 5 7 10
sum_accuracy = 0
kfold = KFold(fold_n, shuffle=True, random_state=42)
enc = LabelEncoder()
fol = []
acc, rc, pr, f1 = [], [], [], []

for train_index, test_index in kfold.split(X):
    fol.append(fold_i)
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    svf = open('output/ResultX.txt', 'w')
    sv_text = '\n'.join(str(item) for item in X_test).replace("   "," ")
    svf.write(sv_text)
    svY = open ('output/y_train.txt', 'w')
    svY.write('\n'.join(str(item) for item in y_train))

    #TFIDF
    tf = TfidfVectorizer(decode_error="replace")
    X_train = tf.fit_transform(X_train)
    X_test = tf.transform(X_test)
    
    # label Encode
    y_train = enc.fit_transform(y_train)
    y_test = enc.transform(y_test)
    
    # Balance
    # smt = SMOTE()
    # X_train, y_train = smt.fit_resample(X_train, y_train)
    # print(y_train.shape)

    # Algorithm
    clf = ModifiedKNN(k_value)
    clf.fit(X_train, y_train)
    pred, jarak = clf.predict(X_test)
    neigbor_index = clf.get_neigbors(X_test)

    # Confusion Matrix
    #tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    accuracy = accuracy_score(y_test, pred)*100
    precision = precision_score(y_test, pred)*100
    recall = recall_score(y_test, pred)*100
    f1_scores = f1_score(y_test, pred)*100
    #accuracy = (tp + tn) / (tp + fp + tn + fn)*100
    #precision = (tp) / (tp + fp)*100
    #recall = (tp) / (tp + fn)*100
    #f1_scores = (2 * precision * recall) / (precision + recall)
    #plot_conf_metrics(y_test, pred)

    sum_accuracy += accuracy
    pred = enc.inverse_transform(pred)

    fold_i += 1
    acc.append(accuracy)
    pr.append(precision)
    rc.append(recall)
    f1.append(f1_scores)

In [128]:
with open("output/MKNN_prediction.txt", "w") as f:
    mknn_predited_label ='\n'.join(str(item) for item in pred)
    f.write(mknn_predited_label)
with open('output/jarak_ttg.txt', 'w') as g:
    jarak = [nMin(k_value,map(float,i)) for i in jarak]
    mknn_distance = '\n'.join(str(ls) for ls in jarak)
    g.write(mknn_distance)
with open('output/index_ttg.txt', 'w') as j:
    j.write('\n'.join(str(a) for a in neigbor_index))

In [129]:
knn_pred = pd.read_csv('output/MKNN_prediction.txt', names=['Sentiment'])
jarak_pred = pd.read_csv('output/jarak_ttg.txt', names=['Distance'], sep='\t')
text_test = pd.read_csv('output/ResultX.txt', names=['text'])
index_pred = pd.read_csv('output/index_ttg.txt', names=['Neigbor'])
text_test = text_test.join(knn_pred)
text_test = text_test.join(jarak_pred)
text_test = text_test.join(index_pred)
#text_test['Sentiment'] = text_test['Sentiment'].apply(lambda x: 'Positive' if x == 1 else 'Negative')
text_test = text_test.dropna()

In [130]:
text_test.head()

,text,Sentiment,Distance,Neigbor
0,kdma teman nonton money heist korea epsiode ga...,Negative,"[1.0858738698060917, 1.128501310566763, 1.1463...",[234 858 3 883 878 853 208 84 52]
1,expectation money heist original,Negative,"[1.0181489878838212, 1.0181489878838212, 1.039...",[311 310 68 309 471 418 129 331 866]
2,senin baca review money heist versi korea pena...,Negative,"[1.0551266547638418, 1.062683298682043, 1.1187...",[402 326 254 161 435 208 106 751 588]
3,weh produksi money heist money heist,Negative,"[1.008169535269051, 1.0202728522824542, 1.0534...",[239 47 222 259 272 127 328 271 409]
4,inget ending film money heist keruk bagiin rak...,Positive,"[1.093625820481299, 1.093625820481299, 1.11537...",[666 650 174 247 98 258 205 374 677]


In [131]:
new_frame = pd.DataFrame(X_test)
new_frame = new_frame.join(knn_pred)

avg_acc = sum_accuracy/fold_n
maxs = max(acc)
mins = min(acc)
res_df = pd.DataFrame({'iterasi':fol, 'Accuracy': acc, 'Precison':pr, 'Recall':rc, 'f1 score':f1})
print("Avearge accuracy : ", str("%.4f" % avg_acc)+'%')
print("Max Score : ",str(maxs),"in Fold : ", str(acc.index(maxs)+1))
print("Min Score : ",str(mins), "in Fold : ", str(acc.index(mins)+1))

Avearge accuracy :  58.6000%
Max Score :  72.0 in Fold :  5
Min Score :  53.0 in Fold :  3


In [132]:
res_df.Accuracy.to_csv('output/res_df.txt', index=False)
res_df.head(11)

,iterasi,Accuracy,Precison,Recall,f1 score
0,1,59.0,62.500000,31.914894,42.253521
1,2,64.0,62.068966,41.860465,50.000000
2,3,53.0,43.396226,57.500000,49.462366
3,4,53.0,38.235294,33.333333,35.616438
4,5,72.0,69.767442,66.666667,68.181818
5,6,56.0,44.117647,37.500000,40.540541
6,7,54.0,44.186047,46.341463,45.238095
7,8,55.0,85.714286,21.818182,34.782609
8,9,59.0,90.909091,20.000000,32.786885
9,10,61.0,50.980392,65.000000,57.142857


In [ ]:
import matplotlib.pyplot as plot

In [ ]:
res_df[['Accuracy', 'Precison', 'Recall', 'f1 score']].plot.line(title="Akurasi tiap Fold")
plot.show(block=True)

# Algorithm K-NN

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.metrics import f1_score, recall_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
k_value = 9 # 1 - 25
X = sentiment_result['text'].values
y = sentiment_result['Sentiment'].values
fold_i = 1
fold_n = 10 # 3 5 7 10
sum_accuracy = 0
kfold = KFold(fold_n, shuffle=True, random_state=42)
enc = LabelEncoder()
fol = []
acc, rc, pr, f1 = [], [], [], []

for train_index, test_index in kfold.split(X):
    fol.append(fold_i)
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    svf = open('output/ResultX.txt', 'w')
    sv_text = '\n'.join(str(item) for item in X_test).replace("   "," ")
    svf.write(sv_text)
    svY = open ('output/y_train.txt', 'w')
    svY.write('\n'.join(str(item) for item in y_train))

    #TFIDF
    tf = TfidfVectorizer(decode_error="replace")
    X_train = tf.fit_transform(X_train)
    X_test = tf.transform(X_test)
    
    y_train = enc.fit_transform(y_train)
    y_test = enc.transform(y_test)

    # Algorithm
    clf = KNeighborsClassifier(k_value)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)

    # Confusion Matrix
    #tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    accuracy = accuracy_score(y_test, pred)*100
    precision = precision_score(y_test, pred)*100
    recall = recall_score(y_test, pred)*100
    f1_scores = f1_score(y_test, pred)*100
    #accuracy = (tp + tn) / (tp + fp + tn + fn)*100
    #precision = (tp) / (tp + fp)*100
    #recall = (tp) / (tp + fn)*100
    #f1_scores = (2 * precision * recall) / (precision + recall)
    #plot_conf_metrics(y_test, pred)

    sum_accuracy += accuracy
    pred = enc.inverse_transform(pred)

    fold_i += 1
    acc.append(accuracy)
    pr.append(precision)
    rc.append(recall)
    f1.append(f1_scores)

In [ ]:
with open("output/MKNN_prediction.txt", "w") as f:
    mknn_predited_label ='\n'.join(str(item) for item in pred)
    f.write(mknn_predited_label)

In [ ]:
knn_pred = pd.read_csv('output/MKNN_prediction.txt', names=['Sentiment'])
text_test = pd.read_csv('output/ResultX.txt', names=['text'])
text_test = text_test.join(knn_pred)
#text_test['Sentiment'] = text_test['Sentiment'].apply(lambda x: 'Positive' if x == 1 else 'Negative')
text_test = text_test.dropna()

In [ ]:
text_test.head()

In [ ]:
new_frame = pd.DataFrame(X_test)
new_frame = new_frame.join(knn_pred)

avg_acc = sum_accuracy/fold_n
maxs = max(acc)
mins = min(acc)
res_df = pd.DataFrame({'iterasi':fol, 'Accuracy': acc, 'Precison':pr, 'Recall':rc, 'f1 score':f1})
print("Avearge accuracy : ", str("%.4f" % avg_acc)+'%')
print("Max Score : ",str(maxs),"in Fold : ", str(acc.index(maxs)+1))
print("Min Score : ",str(mins), "in Fold : ", str(acc.index(mins)+1))

In [ ]:
res_df.to_csv('hs.txt', index=False)
res_df.head(11)